In [1]:
# project retail modeling

In [1]:
pwd()

'C:\\Users\\ujjwa'

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
ld_train=pd.read_csv(r"C:\Users\ujjwa\Downloads\store_train.csv",encoding='cp1252')

In [5]:
ld_test=pd.read_csv(r"C:\Users\ujjwa\Downloads\store_test.csv",encoding='cp1252')

In [6]:
ld_train.columns

Index(['Id', 'sales0', 'sales1', 'sales2', 'sales3', 'sales4', 'country',
       'State', 'CouSub', 'countyname', 'storecode', 'Areaname',
       'countytownname', 'population', 'state_alpha', 'store_Type', 'store'],
      dtype='object')

In [7]:
ld_test['store']=np.nan

In [8]:
ld_test["data"]="test"

In [9]:
ld_train["data"]="train"

In [10]:
ld=pd.concat([ld_test,ld_train],axis=0)

In [11]:
list(zip(ld.columns,ld.dtypes,ld.nunique()))

[('Id', dtype('int64'), 4769),
 ('sales0', dtype('int64'), 493),
 ('sales1', dtype('int64'), 442),
 ('sales2', dtype('int64'), 464),
 ('sales3', dtype('int64'), 626),
 ('sales4', dtype('int64'), 737),
 ('country', dtype('float64'), 329),
 ('State', dtype('int64'), 54),
 ('CouSub', dtype('int64'), 1533),
 ('countyname', dtype('O'), 1962),
 ('storecode', dtype('O'), 2572),
 ('Areaname', dtype('O'), 2572),
 ('countytownname', dtype('O'), 3176),
 ('population', dtype('float64'), 4467),
 ('state_alpha', dtype('O'), 54),
 ('store_Type', dtype('O'), 4),
 ('store', dtype('float64'), 2),
 ('data', dtype('O'), 2)]

In [12]:
ld.drop(["Id"],axis=1,inplace=True)

In [13]:
ld.select_dtypes(["object"]).columns

Index(['countyname', 'storecode', 'Areaname', 'countytownname', 'state_alpha',
       'store_Type', 'data'],
      dtype='object')

In [14]:
ld.drop(['countyname','storecode','Areaname','countytownname'],axis=1,inplace=True)

In [15]:
cat_cols=ld.select_dtypes(["object"]).columns[:-1]

In [16]:
cat_cols

Index(['state_alpha', 'store_Type'], dtype='object')

In [17]:
for col in cat_cols:
    freqs=ld[col].value_counts()
    k=freqs.index[freqs>100][:-1]
    for cat in k:
        name=col+'_'+cat
        ld[name]=(ld[col]==cat).astype(int)
    del ld[col]
    print(col)
    
    

state_alpha
store_Type


In [18]:
ld.isnull().sum()

sales0                             0
sales1                             0
sales2                             0
sales3                             0
sales4                             0
country                            1
State                              0
CouSub                             0
population                         2
store                           1431
data                               0
state_alpha_ME                     0
state_alpha_MA                     0
state_alpha_NH                     0
state_alpha_VT                     0
state_alpha_TX                     0
state_alpha_CT                     0
state_alpha_GA                     0
state_alpha_VA                     0
state_alpha_KY                     0
state_alpha_MO                     0
state_alpha_KS                     0
store_Type_Supermarket Type1       0
store_Type_Grocery Store           0
store_Type_Supermarket Type3       0
dtype: int64

In [19]:
for col in ld.columns:
    if (col not in ['store','data'])& (ld[col].isnull().sum()>0):
        ld.loc[ld[col].isnull(),col]=ld.loc[ld['data']=='train',col].mean()

In [20]:
ld_train=ld[ld["data"]=="train"]

In [21]:
ld_test=ld[ld["data"]=="test"]

In [22]:
ld_test.drop(["store"],axis=1,inplace=True)


C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
ld_train.drop(['data'],axis=1,inplace=True)

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
ld_test.drop(['data'],axis=1,inplace=True)

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
# checking the response on training data

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
train,test=train_test_split(ld_train,test_size=0.2,random_state=2)

In [28]:
x_train=train.drop("store",axis=1)
y_train=train["store"]
x_test=test.drop("store",axis=1)
y_test=test["store"]

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
lm= LogisticRegression()

In [31]:
lm.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
logr=lm.predict(x_test)

In [33]:
from sklearn.metrics import roc_auc_score

In [34]:
roc_auc_score(logr,y_test)

0.6821837727558064

In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
clf = RandomForestClassifier()

In [38]:
param_dist = {"n_estimators":[100,200,300,500,700,1000],
              
              "bootstrap": [True, False],
              'class_weight':[None,'balanced'], 
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }



In [39]:
# run randomized search
n_iter_search = 10
# n_iter parameter of RandomizedSeacrhCV controls, how many 
# parameter combination will be tried; out of all possible given values

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='roc_auc',cv=5)
random_search.fit(x_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [100, 200, 300, 500, 700, 1000], 'bootstrap': [True, False], 'class_weight': [None, 'balanced'], 'criterion': ['entropy', 'gini'], 'max_depth': [None, 5, 10, 15, 20, 30, 50, 70], 'min_samples_leaf': [1, 2, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [41]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [42]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.816 (std: 0.00776)
Parameters: {'n_estimators': 700, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 10, 'criterion': 'gini', 'class_weight': None, 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.816 (std: 0.00618)
Parameters: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_depth': 15, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': False}

Model with rank: 3
Mean validation score: 0.816 (std: 0.00654)
Parameters: {'n_estimators': 700, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_depth': 30, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': False}

Model with rank: 4
Mean validation score: 0.815 (std: 0.00582)
Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20, 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': True}

Model with rank: 5
Mean validation score: 0.814 (std: 0.00644)
Parameters: {

In [45]:
rf=random_search.best_estimator_

In [46]:
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [47]:
from xgboost.sklearn import XGBClassifier

In [48]:
xgb_params = {  
                "learning_rate":[0.01,0.05,0.1,0.3,0.5],
                "gamma":[i/10.0 for i in range(0,5)],
                "max_depth": [2,3,4,5,6,7,8],
                "min_child_weight":[1,2,5,10],
                "max_delta_step":[0,1,2,5,10],
                "subsample":[i/10.0 for i in range(5,10)],
                "colsample_bytree":[i/10.0 for i in range(5,10)],
                "colsample_bylevel":[i/10.0 for i in range(5,10)],
                "reg_lambda":[1e-5, 1e-2, 0.1, 1, 100], 
                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
                "scale_pos_weight":[1,2,3,4,5,6,7,8,9],
                "n_estimators":[100,500,700,1000]
             }


In [49]:
xgb=XGBClassifier(objective='binary:logistic')

In [51]:
n_iter=50

random_search=RandomizedSearchCV(xgb,n_jobs=-1,cv=5,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=xgb_params)

In [52]:
random_search.fit(x_train,y_train)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
          fit_params=None, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5], 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'max_depth': [2, 3, 4, 5, 6, 7, 8], 'min_child_weight': [1, 2, 5, 10], 'max_delta_step': [0, 1, 2, 5, 10], 'subsample': [0.5, 0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9], 'co....1, 1, 100], 'scale_pos_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'n_estimators': [100, 500, 700, 1000]},
          pre_dispatch='2*n_job

In [53]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.815 (std: 0.00614)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 2, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 1000, 'min_child_weight': 2, 'max_depth': 4, 'max_delta_step': 5, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.9}

Model with rank: 2
Mean validation score: 0.814 (std: 0.00710)
Parameters: {'subsample': 0.5, 'scale_pos_weight': 8, 'reg_lambda': 1e-05, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 10, 'max_depth': 6, 'max_delta_step': 5, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.6}

Model with rank: 3
Mean validation score: 0.812 (std: 0.00575)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 1, 'reg_lambda': 0.01, 'reg_alpha': 1e-05, 'n_estimators': 500, 'min_child_weight': 10, 'max_depth': 7, 'max_delta_step': 0, 'learning_rate': 0.01, 'gamma': 0.4, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.8}

Model with rank: 4
Mean 

In [55]:
# here  i have used three model by tuning thier parameter the maximum accuracy we get from random forest classifier so we use 
# random forest among logistic,xgboast and random Forest

In [57]:
logr=rf.predict(x_test)

In [58]:
roc_auc_score(logr,y_test)

0.7828593610719728

In [59]:
# predicting on whole data set

In [61]:
x_train=ld_train.drop("store",axis=1)
y_train=ld_train["store"]


In [62]:
Parameters: {'n_estimators': 700, 
             'min_samples_split': 10, 
             'min_samples_leaf': 1,
             'max_depth': 10, 
             'criterion': 'gini',
             'class_weight': None, 
             'bootstrap': True}

In [63]:
rf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=10, 
                          max_features=10, max_leaf_nodes=None, min_impurity_split=1e-07, 
                          min_samples_leaf=10, min_samples_split=20, min_weight_fraction_leaf=0.0, 
                          n_estimators=700, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False)

In [64]:
rf.fit(x_train,y_train)

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

...

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21.

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=1e-07,
            min_samples_leaf=10, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [65]:
final_model=rf.predict(ld_test)

In [68]:
final_model

array([0., 0., 0., ..., 0., 0., 0.])

In [69]:
ld_test["store"]=final_model

C:\Users\ujjwa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
ld_test=pd.read_csv(r"C:\Users\ujjwa\Downloads\store_test.csv",encoding='cp1252')


In [93]:
ld_test["store"]=final_model

In [94]:
pd.DataFrame(ld_test).to_csv("retail_otput.csv",index=False)